In [ ]:
"""
Created on Thu Feb 08 10:42 2024

Look at some 2D variables in the Casimir runs

Author: @claraburgard
"""

In [36]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'

In [6]:
file_open_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
file_closed_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')

In [7]:
file_open_mean10 = file_open_mean10.where(file_open_mean10['so'] > 0)


In [13]:
lon = file_open_mean10.nav_lon
lat = file_open_mean10.nav_lat

In [30]:
T_30W_closed = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')
T_30W_open = file_open_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')

In [24]:
ref = file_closed_mean10 .where((lon > -31.) & (lon < -29.)).isel(deptht=0)
plt.figure()

llon = lon
llat = lon

proj = ccrs.PlateCarree(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [31]:
(T_30W_closed - T_30W_open).plot()

In [33]:
S_30W_closed = file_closed_mean10['so'].where((lon > -31.) & (lon < -29.)).mean('x')
S_30W_open = file_open_mean10['so'].where((lon > -31.) & (lon < -29.)).mean('x')

In [59]:
depth_closed_30W = file_closed_mean10['e3t'].cumsum('deptht').where((lon > -31.) & (lon < -29.)).mean('x')
depth_open_30W = file_open_mean10['e3t'].cumsum('deptht').where((lon > -31.) & (lon < -29.)).mean('x')

In [60]:
density_closed_30W = gsw.rho(S_30W_closed, T_30W_closed, depth_closed_30W)
density_open_30W = gsw.rho(S_30W_open, T_30W_open, depth_open_30W)

In [61]:
(density_open_30W - density_closed_30W).plot()